In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

# Quiz App

- Given a page of a pdf
- Given a concept
- Simple interface
- Quickly generate flexible quizzes
  - Generate any type of questions (True or False, text based questions, multiple choice)
- Feedback on the answers

In [1]:
from openai import OpenAI

def get_response(prompt):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-4o", 
                             messages=
                             [
                                 {"role": "system", "content": "You are a helpful assistant."},
                                 {"role": "user", "content": prompt}   
                             ],
                             temperature=0.0,
                             n = 1
                             )
    return response.choices[0].message.content


prompt = "Create a quiz about the basics of Python programming."
quiz_python = get_response(prompt)
quiz_python

'Sure! Here\'s a quiz to test your knowledge of the basics of Python programming:\n\n### Python Basics Quiz\n\n#### Question 1:\nWhat is the correct file extension for Python files?\na) .pt  \nb) .py  \nc) .pyt  \nd) .python  \n\n#### Question 2:\nHow do you create a variable with the numeric value 5 in Python?\na) x = 5  \nb) int x = 5  \nc) x := 5  \nd) x == 5  \n\n#### Question 3:\nWhich of the following is the correct syntax to output "Hello, World!" in Python?\na) echo "Hello, World!"  \nb) printf("Hello, World!")  \nc) print("Hello, World!")  \nd) cout << "Hello, World!"  \n\n#### Question 4:\nHow do you insert comments in Python code?\na) // This is a comment  \nb) # This is a comment  \nc) <!-- This is a comment -->  \nd) /* This is a comment */  \n\n#### Question 5:\nWhat is the correct way to create a function in Python?\na) function myFunction():  \nb) def myFunction():  \nc) create myFunction():  \nd) func myFunction():  \n\n#### Question 6:\nWhich of the following is a cor

In [2]:
from IPython.display import Markdown, display

Markdown(quiz_python)

Sure! Here's a quiz to test your knowledge of the basics of Python programming:

### Python Basics Quiz

#### Question 1:
What is the correct file extension for Python files?
a) .pt  
b) .py  
c) .pyt  
d) .python  

#### Question 2:
How do you create a variable with the numeric value 5 in Python?
a) x = 5  
b) int x = 5  
c) x := 5  
d) x == 5  

#### Question 3:
Which of the following is the correct syntax to output "Hello, World!" in Python?
a) echo "Hello, World!"  
b) printf("Hello, World!")  
c) print("Hello, World!")  
d) cout << "Hello, World!"  

#### Question 4:
How do you insert comments in Python code?
a) // This is a comment  
b) # This is a comment  
c) <!-- This is a comment -->  
d) /* This is a comment */  

#### Question 5:
What is the correct way to create a function in Python?
a) function myFunction():  
b) def myFunction():  
c) create myFunction():  
d) func myFunction():  

#### Question 6:
Which of the following is a correct way to create a list in Python?
a) myList = (1, 2, 3)  
b) myList = [1, 2, 3]  
c) myList = {1, 2, 3}  
d) myList = <1, 2, 3>  

#### Question 7:
How do you check the length of a list `myList` in Python?
a) len(myList)  
b) length(myList)  
c) myList.length()  
d) myList.len()  

#### Question 8:
Which of the following is used to handle exceptions in Python?
a) try/except  
b) do/catch  
c) try/catch  
d) handle/except  

#### Question 9:
What is the output of the following code?
```python
print(2 ** 3)
```
a) 5  
b) 6  
c) 8  
d) 9  

#### Question 10:
How do you start a while loop in Python?
a) while x > 0 {  
b) while (x > 0):  
c) while x > 0:  
d) while (x > 0) {  

### Answers:
1. b) .py
2. a) x = 5
3. c) print("Hello, World!")
4. b) # This is a comment
5. b) def myFunction():
6. b) myList = [1, 2, 3]
7. a) len(myList)
8. a) try/except
9. c) 8
10. c) while x > 0:

Feel free to use this quiz to test your understanding of Python basics!

Requirements for the app

1. We have to be specific
   1. We want to specify the number of questions we want
2. We have to be able to feed in contextual information
3. We want to be able to specify the types of questions we want created (T/F, multiple choice, text...)

In [3]:
from langchain.prompts import ChatPromptTemplate

In [4]:
template = """
You are an expert quiz maker for technical fields. Let's think step by step and
create a quiz with {num_questions} {quiz_type} questions about the following concept/content: {quiz_context}.

The format of the quiz could be one of the following:
- Multiple-choice: 
- Questions:
    <Question1>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>
    <Question2>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>
    ....
- Answers:
    <Answer1>: <a|b|c|d>
    <Answer2>: <a|b|c|d>
    ....
    Example:
    - Questions:
    - 1. What is the time complexity of a binary search tree?
        a. O(n)
        b. O(log n)
        c. O(n^2)
        d. O(1)
    - Answers: 
        1. b
- True-false:
    - Questions:
        <Question1>: <True|False>
        <Question2>: <True|False>
        .....
    - Answers:
        <Answer1>: <True|False>
        <Answer2>: <True|False>
        .....
    Example:
    - Questions:
        - 1. What is a binary search tree?
        - 2. How are binary search trees implemented?
    - Answers:
        - 1. True
        - 2. False
- Open-ended:
- Questions:
    <Question1>: 
    <Question2>:
- Answers:    
    <Answer1>:
    <Answer2>:
Example:
    Questions:
    - 1. What is a binary search tree?
    - 2. How are binary search trees implemented?
    
    - Answers: 
        1. A binary search tree is a data structure that is used to store data in a sorted manner.
        2. Binary search trees are implemented using linked lists.
"""

In [5]:
prompt = ChatPromptTemplate.from_template(template)

In [6]:
prompt.format(num_questions=3, quiz_type="multiple-choice", quiz_context="Data Structures in Python Programming")

"Human: \nYou are an expert quiz maker for technical fields. Let's think step by step and\ncreate a quiz with 3 multiple-choice questions about the following concept/content: Data Structures in Python Programming.\n\nThe format of the quiz could be one of the following:\n- Multiple-choice: \n- Questions:\n    <Question1>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>\n    <Question2>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>\n    ....\n- Answers:\n    <Answer1>: <a|b|c|d>\n    <Answer2>: <a|b|c|d>\n    ....\n    Example:\n    - Questions:\n    - 1. What is the time complexity of a binary search tree?\n        a. O(n)\n        b. O(log n)\n        c. O(n^2)\n        d. O(1)\n    - Answers: \n        1. b\n- True-false:\n    - Questions:\n        <Question1>: <True|False>\n        <Question2>: <True|False>\n        .....\n    - Answers:\n        <Answer1>: <True|False>\n        <Answer2>: <True|False>\n        .....\n    Example:\n    - Questions:\n        

In [7]:
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
import re

In [8]:
chain = prompt | ChatOpenAI(temperature=0.0, model="gpt-4o") | StrOutputParser()

In [9]:
quiz_response = chain.invoke({"num_questions":3, "quiz_type":"multiple-choice", "quiz_context":"Data Structures in Python Programming",})

In [10]:
Markdown(quiz_response)

Sure, let's create a multiple-choice quiz about Data Structures in Python Programming. We'll include three questions, each with four possible answers. 

### Questions:

1. Which of the following data structures in Python is mutable and allows duplicate elements?
    a. Tuple
    b. List
    c. Set
    d. Dictionary

2. What is the time complexity of accessing an element in a Python dictionary by its key?
    a. O(n)
    b. O(log n)
    c. O(1)
    d. O(n log n)

3. Which of the following methods is used to add an element to the end of a list in Python?
    a. append()
    b. add()
    c. insert()
    d. extend()

### Answers:

1. b
2. c
3. a

Feel free to use this quiz to test knowledge on data structures in Python programming!

In [11]:
quiz_response = chain.invoke({"num_questions": 4, "quiz_type": "True-false", "quiz_context": "Python programming"})

In [12]:
Markdown(quiz_response)

Sure, let's create a True-false quiz about Python programming. We'll follow the format you provided:

### Questions:
1. Python is a statically typed language.
2. The `print` function in Python automatically adds a newline character at the end of its output.
3. In Python, lists are mutable, but tuples are immutable.
4. Python supports multiple inheritance.

### Answers:
1. False
2. True
3. True
4. True

### Explanation:
1. **False**: Python is a dynamically typed language, meaning that the type of a variable is interpreted at runtime.
2. **True**: By default, the `print` function in Python adds a newline character at the end of its output.
3. **True**: Lists in Python are mutable, meaning their contents can be changed, while tuples are immutable, meaning their contents cannot be changed once they are created.
4. **True**: Python supports multiple inheritance, allowing a class to inherit from more than one base class.

In [14]:
quiz_response = chain.invoke({"num_questions": 2, "quiz_type": "Open-ended", "quiz_context": "Python programming"})

In [15]:
Markdown(quiz_response)

- Questions:
    1. What is Python programming?
    2. How is Python different from other programming languages?
    
- Answers:
    1. Python programming is a high-level, interpreted, and general-purpose programming language known for its simplicity and readability.
    2. Python is different from other programming languages due to its easy syntax, dynamic typing, and extensive standard library.